In [113]:
pip install -r requirements.txt

Python(57350) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [114]:
from llama_index.readers.obsidian import ObsidianReader
from llama_index.core.memory.chat_memory_buffer import MessageRole
from llama_index.core import SimpleDirectoryReader, KnowledgeGraphIndex, VectorStoreIndex
from llama_index.core.graph_stores import SimpleGraphStore
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core import Document, PropertyGraphIndex
from llama_index.core.storage.index_store import SimpleIndexStore
from llama_index.core.vector_stores import SimpleVectorStore
from llama_index.core import Settings
from IPython.display import Markdown, display
from llama_index.llms.ollama import Ollama
from tqdm.notebook import tqdm
import time
import os
from llama_index.core.llms import ChatMessage
from llama_index.core import StorageContext
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core.memory import ChatMemoryBuffer
import logging
import sys
import ipywidgets as widgets
import json
from llama_index.core.callbacks import CallbackManager
from llama_index.core.callbacks import LlamaDebugHandler
from llama_index.core import ServiceContext
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import KnowledgeGraphRAGRetriever
from llama_index.core.indices.property_graph import (
    SimpleLLMPathExtractor,
    SchemaLLMPathExtractor,
    DynamicLLMPathExtractor,
)
import yaml
import networkx as nx
from pyvis.network import Network
from dotenv import load_dotenv

In [115]:
from llama_index.core import (
    load_index_from_storage,
    load_indices_from_storage,
    load_graph_from_storage,
)

In [116]:
import nest_asyncio

nest_asyncio.apply()

In [117]:
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

# Set LLM (OpenAI)

In [118]:
load_dotenv()  # Charge les variables depuis le fichier .env
api_key = os.getenv("OPENAI_API_KEY_UP")
# Modifier ou ajouter une variable d'environnement
os.environ["OPENAI_API_KEY"] = api_key

In [119]:
llm = OpenAI(temperature=0, model="gpt-4o", max_tokens=3000)
Settings.llm = llm
Settings.chunk_size = 512

# Set local LLM for embeddings

In [120]:
# bge-base embedding model
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")
#Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
INFO:sentence_transformers.SentenceTransformer:2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']


# Set LLM for chat  (Local)

In [121]:
#llm = Ollama(model="tinyllama", request_timeout=120.0)
#Settings.llm = llm
#Settings.chunk_size = 512

# Test LLM

In [122]:
messages = [
    ChatMessage(
        role="system", content="You are a data governance consultant"
    ),
    ChatMessage(role="user", content="What's your favorite data tool ?"),
]
resp = llm.chat(messages)
print(resp)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
assistant: As a data governance consultant, I don't have personal preferences or feelings. However, I can provide insights into popular data tools that are widely used in the industry. Some of the most favored tools for data governance include:

1. **Collibra**: Known for its comprehensive data governance capabilities, including data cataloging, data stewardship, and data quality management.

2. **Informatica**: Offers a suite of tools for data integration, data quality, and master data management, which are essential for effective data governance.

3. **Alation**: A leading data ca

# Load storage contexts

## Load vector storage context

In [123]:
vector_storage_context = StorageContext.from_defaults(
    docstore=SimpleDocumentStore.from_persist_dir(persist_dir="vector"),
    vector_store=SimpleVectorStore.from_persist_dir(
        persist_dir="vector"
    ),
    index_store=SimpleIndexStore.from_persist_dir(persist_dir="vector"),
)

## Load knowledge graph storage context

In [124]:
graph_storage_context = StorageContext.from_defaults(
    docstore=SimpleDocumentStore.from_persist_dir(persist_dir="knowledge_graph"),
    graph_store=SimpleGraphStore.from_persist_dir(
        persist_dir="knowledge_graph"
    ),
    index_store=SimpleIndexStore.from_persist_dir(persist_dir="knowledge_graph"),
)

## Load onto graph storage context


In [125]:
onto_storage_context = StorageContext.from_defaults(
    docstore=SimpleDocumentStore.from_persist_dir(persist_dir="onto_graph"),
    graph_store=SimpleGraphStore.from_persist_dir(
        persist_dir="onto_graph"
    ),
    index_store=SimpleIndexStore.from_persist_dir(persist_dir="onto_graph"),
)

In [126]:
print(onto_storage_context)

StorageContext(docstore=<llama_index.core.storage.docstore.simple_docstore.SimpleDocumentStore object at 0x150de6410>, index_store=<llama_index.core.storage.index_store.simple_index_store.SimpleIndexStore object at 0x2973e1660>, vector_stores={'default': SimpleVectorStore(stores_text=False, is_embedding_query=True, data=SimpleVectorStoreData(embedding_dict={}, text_id_to_ref_doc_id={}, metadata_dict={})), 'image': SimpleVectorStore(stores_text=False, is_embedding_query=True, data=SimpleVectorStoreData(embedding_dict={}, text_id_to_ref_doc_id={}, metadata_dict={}))}, graph_store=<llama_index.core.graph_stores.simple.SimpleGraphStore object at 0x2973e1600>, property_graph_store=None)


# Load index

## Load vector index

In [127]:
simple_index = load_index_from_storage(vector_storage_context)

INFO:llama_index.core.indices.loading:Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.


### Set retriever

In [128]:
simple_query_engine = simple_index.as_query_engine(
 include_text=True,
 response_mode="tree_summarize",
 embedding_mode="hybrid",
 similarity_top_k=8,
)

### Test retriever

In [129]:
simple_rag_retriever = simple_index.as_retriever(
    retriever_mode="hybrid",  # or "embedding" or "hybrid"
    verbose=True
)

response = simple_query_engine.query(
    "What is open model ?",
)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [130]:
display(Markdown(f"<b>{response}</b>"))

<b>An open model is a mode of organization and knowledge production that relies on digital technology, the free circulation of information, and collaboration on an Internet scale. These models aim to benefit from the interactions between ecosystems and are characterized by shared governance. They encompass various fields such as open science, open education, open software, open data, and open hardware. Open models enable the co-production of digital commons, which can be freely used, studied, modified, and redistributed. They are part of a broader movement that seeks to understand the socio-technical environments built around digital commons and the benefits and risks of these interactions.</b>

## Load graph index

In [131]:
graph_index = load_index_from_storage(graph_storage_context)

INFO:llama_index.core.indices.loading:Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.


In [132]:
nx_graph = graph_index.get_networkx_graph()

In [133]:
# Count the number of nodes
num_nodes = len(nx_graph.edges())

print(f"Number of nodes in the knowledge graph: {num_nodes}")

Number of nodes in the knowledge graph: 97


In [134]:
g = graph_index.get_networkx_graph()
print(g)
print("Nodes:", g.nodes())
print("Edges:", g.edges())
net = Network(notebook=True, cdn_resources="in_line", directed=True)
net.from_nx(g)

with open("knowledge_graph.html", "w", encoding="utf-8") as f:
    f.write(net.generate_html())

Graph with 150 nodes and 97 edges
Nodes: ['Open artificial intelligence', 'Open models', 'Immature', 'Parent', 'Idea', 'Dream', 'Collective work', 'Open education', 'Removing barriers to education', 'Open educational resources', 'Training', 'The open university', 'Removing entry requirements', '1969', 'Protean', 'It professionals', 'Open education global network', 'Over 250 members', 'Unesco', 'Legal affairs recommendation', 'Open science', 'Isaac pitman', 'First distance learning courses', 'Digital', 'Relationship with knowledge', 'Communities', 'Co-produce digital commons', 'Open source software', 'Foundations of digital universe', 'History of open models', 'Digital evolution', 'Software world', 'Other fields of activity', 'Richard stallman', 'Gnu project', 'Free software movement', 'Reliability', 'Linus torvalds', 'Code on internet', 'Code', 'Gnu general public license', '1989', 'Open license', 'Computer scientists', 'Technical infrastructures', 'Wikipedia', 'Wiki technology', 'Age 

### Set retriever

In [135]:
graph_query_engine = graph_index.as_query_engine(
 include_text=True,
 response_mode="tree_summarize",
 embedding_mode="hybrid",
 similarity_top_k=8,
)

### Test retriever

In [136]:
graph_rag_retriever = graph_index.as_retriever(
    retriever_mode="hybrid",  # or "embedding" or "hybrid"
    verbose=True
)

response = graph_query_engine.query(
    "What is open model ?",
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: ae053f60-f18a-4a7e-9f52-30349bb5f04a: Federating an ecosystem of stakeholders

To carry out this education work on ...
> Querying with idx: ae053f60-f18a-4a7e-9f52-30349bb5f04a: Federating an ecosystem of stakeholders

To carry out this education work on ...
> Querying with idx: ae053f60-f18a-4a7e-9f52-30349bb5f04a: Federating an ecosystem of stakeholders

To carry out this education work on ...
> Querying with idx: ae053f60-f18a-4a7e-9f52-30349bb5f04a: Federating an ecosystem of stakeholders

To carry out this education work on ...
> Querying with idx: ae053f60-f18a-4a7e-9f52-30349bb5f04a: Federating an ecosystem of stakeholders

To carry out this education work on ...
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: f08077e9-166e-43a7-a91d-e7f4a09b00f4: ---
parent: "[[Open Models]]"
---

Open Artificial Intelligence.
> Querying with idx: f08077e9-166e-43a7-a91d-e7f4a09b00f4: ---

In [137]:
display(Markdown(f"<b>{response}</b>"))

<b>Open models are a concept that involves collaborative and open sharing of resources, particularly in the digital realm. They are considered a driving force in society, contributing to the digital revolution and impacting various fields such as education, industry, and science. Open models are characterized by their collective nature and are seen as a part of the broader movement towards open science and open education. They are still in varying stages of maturity but are integral to the architecture of systems that benefit from open collaboration and shared knowledge.</b>

## Load onto index

In [138]:
onto_storage_context = StorageContext.from_defaults(persist_dir="onto_graph")
# Load the PropertyGraphIndex from the storage context

onto_index = load_index_from_storage(onto_storage_context)

INFO:llama_index.core.indices.loading:Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.


In [139]:
onto_index.property_graph_store.save_networkx_graph(
    name="OntoGraph.html"
)

### Set retriever

In [140]:
onto_query_engine = onto_index.as_query_engine(
 include_text=True,
 similarity_top_k=8,
)

### Test retriever

In [141]:
onto_rag_retriever = onto_index.as_retriever(
    retriever_mode="hybrid",  # or "embedding" or "hybrid"
    verbose=True
)

response = onto_query_engine.query(
    "What is open model ?",
)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [142]:
display(Markdown(f"<b>{response}</b>"))

<b>An open model is a concept based on the free circulation of information and collaboration on an Internet scale. It involves studying socio-technical environments to understand the benefits and risks of interactions. Open models should be shared and are built on digital foundations. They aim to democratize emerging concepts and enable citizens to master digital knowledge. These models are part of a broader effort to create a base of knowledge that is accessible to anyone, developed using open software tools, and intended to foster learning and collaboration across various disciplines.</b>

# Visualize knowledge graph

In [143]:
nx_graph = graph_index.get_networkx_graph()

In [144]:
# Count the number of nodes
num_nodes = len(nx_graph.edges())

print(f"Number of nodes in the knowledge graph: {num_nodes}")

Number of nodes in the knowledge graph: 97


In [145]:
g = graph_index.get_networkx_graph()
print(g)
print("Nodes:", g.nodes())
print("Edges:", g.edges())
net = Network(notebook=True, cdn_resources="in_line", directed=True)
net.from_nx(g)

with open("knowledge_graph.html", "w", encoding="utf-8") as f:
    f.write(net.generate_html())

Graph with 150 nodes and 97 edges
Nodes: ['Open artificial intelligence', 'Open models', 'Immature', 'Parent', 'Idea', 'Dream', 'Collective work', 'Open education', 'Removing barriers to education', 'Open educational resources', 'Training', 'The open university', 'Removing entry requirements', '1969', 'Protean', 'It professionals', 'Open education global network', 'Over 250 members', 'Unesco', 'Legal affairs recommendation', 'Open science', 'Isaac pitman', 'First distance learning courses', 'Digital', 'Relationship with knowledge', 'Communities', 'Co-produce digital commons', 'Open source software', 'Foundations of digital universe', 'History of open models', 'Digital evolution', 'Software world', 'Other fields of activity', 'Richard stallman', 'Gnu project', 'Free software movement', 'Reliability', 'Linus torvalds', 'Code on internet', 'Code', 'Gnu general public license', '1989', 'Open license', 'Computer scientists', 'Technical infrastructures', 'Wikipedia', 'Wiki technology', 'Age 

# (Simple) Query the vectors

In [146]:
query = "What is open model ?"
query_engine = simple_index.as_query_engine(
 include_text=True,
 response_mode="tree_summarize",
 embedding_mode="hybrid",
 similarity_top_k=8,
)

response = query_engine.query(query)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [147]:
display(Markdown(f"<b>{response}</b>"))

<b>An open model is a mode of organization and knowledge production that relies on digital technology, the free circulation of information, and collaboration on an Internet scale. These models aim to benefit from the interactions between ecosystems and are characterized by shared governance. They encompass various fields such as open science, open education, open software, open data, and open hardware. Open models enable the co-production of digital commons, which can be freely used, studied, modified, and redistributed. They are part of a broader movement that seeks to understand the socio-technical environments built around digital commons and the benefits and risks of these interactions.</b>

# (Simple) Query the knowledge graph 

In [148]:
query = "What is open model ?"
graph_query_engine = graph_index.as_query_engine(
 include_text=True,
 response_mode="tree_summarize",
 embedding_mode="hybrid",
 similarity_top_k=8,
)

response = graph_query_engine.query(query)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: ae053f60-f18a-4a7e-9f52-30349bb5f04a: Federating an ecosystem of stakeholders

To carry out this education work on ...
> Querying with idx: ae053f60-f18a-4a7e-9f52-30349bb5f04a: Federating an ecosystem of stakeholders

To carry out this education work on ...
> Querying with idx: ae053f60-f18a-4a7e-9f52-30349bb5f04a: Federating an ecosystem of stakeholders

To carry out this education work on ...
> Querying with idx: ae053f60-f18a-4a7e-9f52-30349bb5f04a: Federating an ecosystem of stakeholders

To carry out this education work on ...
> Querying with idx: ae053f60-f18a-4a7e-9f52-30349bb5f04a: Federating an ecosystem of stakeholders

To carry out this education work on ...
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: f08077e9-166e-43a7-a91d-e7f4a09b00f4: ---
parent: "[[Open Models]]"
---

Open Artificial Intelligence.
> Querying with idx: f08077e9-166e-43a7-a91d-e7f4a09b00f4: ---

In [149]:
display(Markdown(f"<b>{response}</b>"))

<b>Open models are a concept that involves collaborative and open sharing of resources, particularly in the digital realm. They are considered a driving force in society, contributing to the digital revolution and impacting various fields such as education, industry, and science. Open models are characterized by their collective nature and are seen as a part of the broader movement towards open science and open education. They are still in varying stages of maturity but are integral to the architecture of systems that benefit from open collaboration and shared knowledge.</b>

# (Simple) Query the onto graph 

In [150]:
query = "What is open model ?"
onto_query_engine = onto_index.as_query_engine(
 include_text=True,
 similarity_top_k=2,
)

response = onto_query_engine.query(query)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [151]:
display(Markdown(f"<b>{response}</b>"))

<b>An open model is a concept based on the free circulation of information and collaboration on an Internet scale. It involves studying socio-technical environments to understand the benefits and risks of interactions. Open models should be shared and are designed to evolve over long periods, potentially spanning decades to centuries. They aim to disseminate knowledge through educational efforts, acculturate citizens, and build a digital commons while federating an ecosystem of stakeholders. The understanding of open models increases comfort in contributing to them, and contributions can be made according to individual skills and means.</b>

## (Node retriever)

In [152]:
retriever = onto_index.as_retriever(
    include_text=True,  # include source text, default True
)

nodes = retriever.retrieve("What is open model ?")


for node in nodes:
    print(node.text)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Here are some facts extracted from the provided text:

Open models -> BASED_ON -> digital
Open models -> STUDY -> socio-technical environments
Open models -> UNDERSTAND -> benefits and risks of interactions
Open models -> SHOULD_BE -> shared
Open models -> BASED_ON -> free circulation of information
Open models -> BASED_ON -> collaboration on an Internet scale

Federating an ecosystem of stakeholders

To carry out this education work on open models and build up this base of knowledge, as well as recursively federating players, it will be important to work on setting up an ecosystem 

# Have a real chat with your data

## Set up the engines

### Vector engines

In [153]:
memory = ChatMemoryBuffer.from_defaults(token_limit=3900)
vector_chat_engine = simple_index.as_chat_engine(
    chat_mode="condense_plus_context",
    memory=memory,
    llm=llm,
    context_prompt=(
        "You are a specialist about open models and you nurture a knowledge base on the subject"
        "Be precise."
        "."
    ),
    verbose=False,
)

### Graph engines

In [154]:
memory = ChatMemoryBuffer.from_defaults(token_limit=3900)
graph_chat_engine = graph_index.as_chat_engine(
    chat_mode="condense_plus_context",
    memory=memory,
    llm=llm,
    verbose=False,
)

### Onto engines

In [155]:
#onto_chat_engine = onto_index.query_engine(
#    chat_mode="condense_plus_context",
#    llm=llm
#)

## Generate

In [156]:
graph_chat_engine.reset()

In [157]:
memory = ChatMemoryBuffer.from_defaults(token_limit=3900)
graph_chat_engine = graph_index.as_chat_engine(
    chat_mode="condense_plus_context",
    memory=memory,
    llm=llm,
    context_prompt=(
        "You are a specialist about open models and you nurture a knowledge base on the subject"
        "Be precise on concepts."
        " "
    ),
    verbose=False,
)

In [158]:
response_stream = graph_chat_engine.stream_chat("""Based on your knowledge and your vision of the world, create a description of Open Access. It needs 5 parts : a definition, a presentation, a usage description, a landscape and history. """)

INFO:llama_index.core.chat_engine.condense_plus_context:Condensed question: Based on your knowledge and your vision of the world, create a description of Open Access. It needs 5 parts : a definition, a presentation, a usage description, a landscape and history. 
Condensed question: Based on your knowledge and your vision of the world, create a description of Open Access. It needs 5 parts : a definition, a presentation, a usage description, a landscape and history. 
Condensed question: Based on your knowledge and your vision of the world, create a description of Open Access. It needs 5 parts : a definition, a presentation, a usage description, a landscape and history. 
Condensed question: Based on your knowledge and your vision of the world, create a description of Open Access. It needs 5 parts : a definition, a presentation, a usage description, a landscape and history. 
Condensed question: Based on your knowledge and your vision of the world, create a description of Open Access. It ne

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 6f14a825-36e2-4c54-a714-6cd3d582a4b2: > _Open Science is the practice of science in such a way that others can coll...
> Querying with idx: 6f14a825-36e2-4c54-a714-6cd3d582a4b2: > _Open Science is the practice of science in such a way that others can coll...
> Querying with idx: 6f14a825-36e2-4c54-a714-6cd3d582a4b2: > _Open Science is the practice of science in such a way that others can coll...
> Querying with idx: 6f14a825-36e2-4c54-a714-6cd3d582a4b2: > _Open Science is the practice of science in such a way that others can coll...
> Querying with idx: 6f14a825-36e2-4c54-a714-6cd3d582a4b2: > _Open Science is the practice of science in such a way that others can coll...
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: d6c00799-f94e-4c3a-88cc-8ddabd8782fd: History of open science

> _The Internet has fundamentally changed the practi...
> Querying with idx: d6c00799-f94e-4c3a-88cc-8

In [159]:
generate = response_stream.print_response_stream()


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
### Definition
Open Access (OA) refers to the practice of providing unrestricted online access to scholarly research outputs, such as journal articles, conference papers, theses, and book chapters. The primary goal of Open Access is to make academic research freely available to anyone with internet access, removing financial, legal, and technical barriers to the dissemination of knowledge. This approach is intended to enhance the visibility, accessibility, and impact of research findings.

### Presentation
Open Access is typically categorized into two main types: "Gold" and "Green."

In [160]:
display(Markdown(f"<b>{generate}</b>"))

<b>None</b>

In [161]:
section_modele = """ 


"""

In [162]:
response_stream = graph_chat_engine.stream_chat("""Based on your knowledge and your vision of the world, create a description of Open Business. It needs 5 parts : a definition, a presentation, a usage description, a landscape and history. Imitate the writing style of : {section_modele}""")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.chat_engine.condense_plus_context:Condensed question: Based on your knowledge and your vision of the world, create a description of Open Business. It needs 5 parts: a definition, a presentation, a usage description, a landscape, and history. Imitate the writing style of the Open Access description provided earlier.
Condensed question: Based on your knowledge and your vision of the world, create a description of Open Business. It needs 5 parts: a definition, a presentation, a usage description, a landscape, and history. Imitate the writing style of the Open Acce

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: ae053f60-f18a-4a7e-9f52-30349bb5f04a: Federating an ecosystem of stakeholders

To carry out this education work on ...
> Querying with idx: ae053f60-f18a-4a7e-9f52-30349bb5f04a: Federating an ecosystem of stakeholders

To carry out this education work on ...
> Querying with idx: ae053f60-f18a-4a7e-9f52-30349bb5f04a: Federating an ecosystem of stakeholders

To carry out this education work on ...
> Querying with idx: ae053f60-f18a-4a7e-9f52-30349bb5f04a: Federating an ecosystem of stakeholders

To carry out this education work on ...
> Querying with idx: ae053f60-f18a-4a7e-9f52-30349bb5f04a: Federating an ecosystem of stakeholders

To carry out this education work on ...
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: c72fcc46-0a54-4e7b-9064-960b40a60531: Importance of open models

All open models are immature and their degree of m...
> Querying with idx: c72fcc46-0a54-4e7b-9064-9

In [163]:
generate = response_stream.print_response_stream()


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
### Définition
L'Open Business désigne un modèle d'entreprise qui repose sur la transparence, la collaboration et le partage ouvert d'informations, de ressources et de processus. Contrairement aux pratiques commerciales traditionnelles qui privilégient la confidentialité et la protection des actifs intellectuels, l'Open Business encourage l'ouverture et l'engagement avec des parties prenantes externes, y compris les clients, les partenaires et même les concurrents. L'objectif est de stimuler l'innovation, d'améliorer l'efficacité et de créer de la valeur partagée.

### Présentation


In [164]:
response_stream = graph_chat_engine.stream_chat("""Based on your knowledge and your vision of the world, create a description of Open Banking. It needs 5 parts : a definition, a presentation, a usage description, a landscape and history. Imitate the writing style of : {section_modele}""")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.chat_engine.condense_plus_context:Condensed question: Based on your knowledge and your vision of the world, create a description of Open Banking. It needs 5 parts: a definition, a presentation, a usage description, a landscape, and history. Please imitate the writing style of the previous sections provided.
Condensed question: Based on your knowledge and your vision of the world, create a description of Open Banking. It needs 5 parts: a definition, a presentation, a usage description, a landscape, and history. Please imitate the writing style of the previous se

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 5276c780-50eb-49c6-92c1-e79022abed9e: ---
url: https://github.com/OpenDesign-WorkingGroup/Open-Design-Definition/bl...
> Querying with idx: 5276c780-50eb-49c6-92c1-e79022abed9e: ---
url: https://github.com/OpenDesign-WorkingGroup/Open-Design-Definition/bl...
> Querying with idx: 5276c780-50eb-49c6-92c1-e79022abed9e: ---
url: https://github.com/OpenDesign-WorkingGroup/Open-Design-Definition/bl...
> Querying with idx: 5276c780-50eb-49c6-92c1-e79022abed9e: ---
url: https://github.com/OpenDesign-WorkingGroup/Open-Design-Definition/bl...
> Querying with idx: 5276c780-50eb-49c6-92c1-e79022abed9e: ---
url: https://github.com/OpenDesign-WorkingGroup/Open-Design-Definition/bl...
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: ae053f60-f18a-4a7e-9f52-30349bb5f04a: Federating an ecosystem of stakeholders

To carry out this education work on ...
> Querying with idx: ae053f60-f18a-4a7e-9f52-3

In [165]:
generate = response_stream.print_response_stream()


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
### Définition
L'Open Banking désigne un système bancaire qui permet le partage sécurisé des données financières des clients entre différentes institutions financières et prestataires de services tiers, avec le consentement explicite des clients. Ce modèle vise à favoriser l'innovation et la concurrence dans le secteur financier en permettant le développement de nouveaux services et produits financiers personnalisés, tout en garantissant la protection des données personnelles.

### Présentation
L'Open Banking repose sur l'utilisation d'interfaces de programmation d'applications (API

In [166]:
response_stream = graph_chat_engine.stream_chat("""Based on your knowledge and your vision of the world, create a description of Open Design. It needs 5 parts : a definition, a presentation, a usage description, a landscape and history. Imitate the writing style of : {section_modele}""")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.chat_engine.condense_plus_context:Condensed question: Based on your knowledge and your vision of the world, create a description of Open Design. It needs 5 parts: a definition, a presentation, a usage description, a landscape, and history. Please imitate the writing style of the previous sections provided.
Condensed question: Based on your knowledge and your vision of the world, create a description of Open Design. It needs 5 parts: a definition, a presentation, a usage description, a landscape, and history. Please imitate the writing style of the previous sect

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 5276c780-50eb-49c6-92c1-e79022abed9e: ---
url: https://github.com/OpenDesign-WorkingGroup/Open-Design-Definition/bl...
> Querying with idx: 5276c780-50eb-49c6-92c1-e79022abed9e: ---
url: https://github.com/OpenDesign-WorkingGroup/Open-Design-Definition/bl...
> Querying with idx: 5276c780-50eb-49c6-92c1-e79022abed9e: ---
url: https://github.com/OpenDesign-WorkingGroup/Open-Design-Definition/bl...
> Querying with idx: 5276c780-50eb-49c6-92c1-e79022abed9e: ---
url: https://github.com/OpenDesign-WorkingGroup/Open-Design-Definition/bl...
> Querying with idx: 5276c780-50eb-49c6-92c1-e79022abed9e: ---
url: https://github.com/OpenDesign-WorkingGroup/Open-Design-Definition/bl...


In [167]:
generate = response_stream.print_response_stream()


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
### Définition
L'Open Design est une approche de conception qui prône la transparence, la collaboration et le partage libre des plans, des idées et des processus de création. Contrairement aux méthodes de conception traditionnelles qui protègent les droits de propriété intellectuelle, l'Open Design encourage la diffusion ouverte des informations pour permettre à d'autres de reproduire, modifier et améliorer les créations. Ce modèle vise à démocratiser l'accès à la conception et à stimuler l'innovation collective.

### Présentation
L'Open Design se manifeste à travers des plateformes

In [168]:
display(Markdown(f"<b>{generate}</b>"))

<b>None</b>

## Sum-up

In [169]:
chat_history = memory.get_all()

# Assuming chat_history is available and contains your messages
assistant_messages = [
    message.content 
    for message in chat_history 
    if message.role == MessageRole.ASSISTANT  # Compare with the enum directly
]


output_filename = r"/Users/arthursarazin/Documents/coreandgraphs/graphandopenmodels/output.md"
# Write to a Markdown file
with open(output_filename, "w", encoding="utf-8") as f:
    for msg in assistant_messages:
        f.write(msg + "\n\n") 